In [0]:
src_path="dbfs:/databricks-datasets/weather/high_temps"

In [0]:
df_weather=spark.read.format("csv").option("header", True).load(src_path)
df_weather.display()

In [0]:
from pyspark.sql.functions import col
